TSG115 - SQL Server on Linux security log translator
====================================================

Create logger.ini
-----------------

This notebook is designed to parse the logs generated by the
secuirty.ldap and security.kerberos loggers for SQL Server on Linux. To
enable these loggers, place the lines below in
*/var/opt/mssql/logger.ini* on the machine running SQL Server on Linux.
Note: this file is case sensitive.

    [Output:security]
    Type = File
    Filename = /var/opt/mssql/log/security.log
    Maxrolloverfiles = 3
    Maxfilesizemb = 1

    [Logger]
    Level = Silent

    [Logger:security.kerberos]
    Level = Error
    Outputs = security

    [Logger:security.ldap]
    Level = Error
    Outputs = security

When errors are hit in the Kerberos or LDAP code paths (used for AD
authentication), logs will be output to /var/opt/mssql/log/security.log.

Reproduce the AD issue
----------------------

Now that the logs are enabled, reproduce the AD issue. This will
generate new errors in /var/opt/mssql/log/security.log

Copy logs
---------

Copy /var/opt/mssql/log/security.log from the SQL Server machine to a
local path.

### Parameters

In [ ]:
filePath = ""

Translate error messages
------------------------

Run the code block below and pass the path to the log file you just
coppied to your machine. The script will parse the log file and find
errors with known solutions.

In [ ]:
# Parse log files
#
import re
from IPython.display import display, Markdown

errorDict = {
    "Key table entry not found$":
        "Missing entry in keytab or keytab is not accessible. Check the file is readable by the user running SQL Server (e.g. mssql) and that"
        " all SPN and UPN entries are present (klist -kte /var/opt/mssql/secrets/mssql.keytab).",
    "No credentials were supplied, or the credentials were unavailable or inaccessible$":
        "Delegation is not enabled or the keytab is missing SPNs or inaccessible. If this is a linked server query, check with your domain"
        " administrator that delegation is enabled for the account that owns the MSSQLSvc SPNs. If this is from a login that failed, check"
        " the keytab is accessible by the user running SQL Server (mssql) and that all SPN and UPN entries are present (klist -kte"
        " /var/opt/mssql/secrets/mssql.keytab).",
    "No key table entry found for [^ ]+@[^ ]+$":
        "Entries are missing from the keytab. Ensure the entry specified in the error is present in SQL's keytab or that mssql-conf is set to"
        " use another principal and the password is valid.",
    "Request ticket server [^ ]+/[^ ]+@[^ ]+ not found in keytab \(ticket kvno [0-9]+\)":
        "SPN entries are missing from the keytab. Ensure the entry specified in the error is present in SQL's keytab with the correct KVNO.",
    "Not setting up the Kerberos credential cache manager as this machine is not domain joined":
        "Machine does not appear domain joined to SQL Server. Ensure the keytab is readable by the user SQL Server is running as (e.g. mssql),"
        " mssql.conf points to the correct keytab, and a default realm is set in /etc/krb5.conf.",
    "Request ticket server [^ ]+/[^ ]+@[^ ]+ kvno [0-9]+ found in keytab but not with enctype [^ ]+":
        "SPN entries are missing from the keytab. Ensure the entry specified in the error is present in SQL's keytab with the requested KVNO"
        " and encryption types. Alternatively, ask the domain administrator to enable the encryption types in your keytab for the account owning"
        " the SPN in the error message.",
    "Request ticket server [^ ]+/[^ ]+@[^ ]+ kvno [0-9]+ enctype [^ ]+ found in keytab but cannot decrypt ticket":
        "SPN entry is malformed or has incorrect password. Validate the password for the SPN specified in the error is correct. If this is an"
        " AES key, make sure that ktutil was not used to create it as ktutil has a bug when creating AES keys for AD",
    "Could not perform RDNS lookup for host '[^ ]+' due to error":
        "Invalid DNS setup. Check that reverse DNS records exist for the domain controllers.",
    "FQDN not returned by RDNS lookup.$":
        "Invalid DNS setup. Check that reverse DNS records exist for the domain controller and they resolve to the fully qualified domain name.",
    "Could not look up short domain name due to error":
        "Invalid DNS setup. Check that the short domain name (e.g. CONTOSO) resolves to the IP of the domain controller.",
    "Failed to bind to LDAP server":
        "Invalid domain controller. Check that DNS and reverse DNS entries for all domain controllers are valid. Also, check that any domain"
        " controllers specified in /etc/krb5.conf are valid (if none, that is ok).",
}

def FindErrorCause(errorString) :
    res = None

    for knownErrorRegex, cause in errorDict.items():
        if re.search(knownErrorRegex, errorString):
            res = cause
            break

    return res

def ParseLogFile() :
    foundError = False

    with open(filePath, 'r') as fp:
        lineNum = 0

        while True:
            line = fp.readline()
            lineNum += 1

            if not line:
                break

            # Find if there is a Kerberos or LDAP error
            #
            errorMatch = re.search('Error \[security\.(kerberos|ldap)\] \<.*\>', line)

            if errorMatch:
                errorString = line[errorMatch.end() + 1:].rstrip()
                cause = FindErrorCause(errorString)

                if cause:
                    display(Markdown('Line {0}: {1}  \n{2}'.format(lineNum, errorString, cause)))
                    foundError = True

    if not foundError:
        print("No translatable errors found in the log.")

print("Log file location:")

if filePath == "":
    filePath = input().strip('"')
else:
    print("")

print(filePath)
ParseLogFile()

Next steps
----------

The errors reported above are the best translations of internal error
messages. If no translatable errors were found, check that the AD login
was added to SQL Server, your client is logged in to AD (kinit on
Linux), and you are attempting to connect to the fully qualified domain
name of the SQL Server instance, not the IP. For more help, visit
[Tutorial: Use Active Directory authentication with SQL Server on
Linux](https://docs.microsoft.com/en-us/sql/linux/sql-server-linux-active-directory-authentication?view=sql-server-2017)

In [ ]:
print("Notebook execution is complete.")